In [39]:
import pandas as pd
import arff
import os 
import re
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [40]:
# Adjust pandas display options to show all values
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [41]:
# Import metadata 
dimension=pd.read_csv('dimension/dimension.csv',delimiter=';')
sector=pd.read_csv('dimension/sector_dimension.csv')

In [42]:
# Clean the .arff file:
# Step 1: Clean random letters appearing and convert them to m
# Step 2: Clean the letter m in the Sector column which should be numeric,
# we know it`s the last column, convert m to ?
# which is the null value for numeric attributes in .arff files
# Step 3: Remove rows where the country Italy appears inb the first column

for filename in os.listdir('data/'):# Loop through each file in the directory
    with open(f'data/{filename}', 'r') as rf: # Open the .arff file
        # Read lines from the file
        lines = rf.readlines()

    for i, line in enumerate(lines):
        
        # Step 1: 
        # Clean random letters appearing and convert them to m
        pattern = r',([a-z]),'# Regex to match comma + single lowercase letter + comma
        # Perform the substitution
        lines[i] = re.sub(pattern, lambda x: ',m,', line)
        
        # Step 2:
        # If there is an m in the last three characters
        if 'm' in line[-3:]:
            # If yes, replace 'm' with '?' in the last 3 characters
            lines[i] = line[:-3] + line[-3:].replace('m', '?')
    
        # Step 3:
        # Split the string by commas
        string_elements = line.split(',')
        # Get the first value
        first_value = string_elements[0]

        # Check if the first value is Italy
        if first_value == 'Italy':
            # If not, mark the line with 'Remove'
            lines[i] = 'Remove'
            
    # Let`s remove all the lines=='Remove'
    lines = [line for line in lines if line != 'Remove']     
    
    # Write modified lines back to a new ARFF file
    with open(f'data_modified/{filename}', 'w') as wf:
        wf.writelines(lines)

In [43]:
# Imports the cleaned up arff files from data_modified and puts them in a single df.

# Initialize an empty list to store DataFrames
dfs_list = []

for filename in os.listdir('data_modified/'):# Loop through each file in the directory
    with open(f'data_modified/{filename}', 'r') as f: # Open the .arff file
    
        raw_data = arff.load(f) # dict obj
        # Store arff in a pandas df
        df = pd.DataFrame(raw_data['data'],columns=[x[0] for x in raw_data['attributes']])
        
        # Create a dict with the mappings
        mappings = dict(zip(dimension['Variable Name'], dimension['Description']))
        # Rename columns in df using the mapping
        df.rename(columns=mappings,inplace=True)

        # Store year and quarter cols
        df['Year'] = int(filename[:4])
        df['Quarter'] = filename[5:7]
        
        # Append DataFrame to the list
        dfs_list.append(df)
        
# Concatenate all DataFrames in the list
df = pd.concat(dfs_list, ignore_index=True)

In [44]:
# Handle missing values
df.replace('m', None, inplace=True)

In [45]:
# Remove rows with no county
df=df.dropna(subset=['Country']).reset_index(drop=True)
# Remove rows with no sector
df=df.dropna(subset=['sectors']).reset_index(drop=True)

In [46]:
# Convert financial indicator cols into floats
cols=df.drop(['Country','Year','Quarter','sectors'],axis=1).columns
df[cols] = df[cols].astype(float)

In [47]:
# Remove rows where more than threshold percent of cols are null
threshold = int(0.7 * len(df.columns))
df=df.dropna(thresh=threshold).reset_index(drop=True)

In [48]:
# Calculate the percentage of NaN values for each column
nan_percentage = (df.isna().mean() * 100)
# Display the percentage of NaN values for each column
print("Percentage of NaN values for each column:")
print(nan_percentage)

Percentage of NaN values for each column:
Country                                                                                                           0.000000
Net profit/total assets                                                                                           0.000000
Total liabilities/total assets                                                                                    0.000000
Working capital/total assets                                                                                      0.190311
Current assets/short-term liabilities                                                                             0.501730
                                                                                                                   ...    
Net cash flow from (used in) operating activities (n)/Net cash flow from (used in) operating activities (n−1)    39.792388
Net cash flow(n)/net cash flow (n−1)                                                             

In [49]:
# Get the sector mapping in, we choose inner to get rid of a sector=0 row
df=df.merge(sector,how='inner', left_on='sectors', right_on='code_sector').drop(['sectors'],axis=1)

In [50]:
# Extract numeric part from 'Quarter' column and convert to float
df['Quarter'] = df['Quarter'].str.extract('(\d+)').astype(float)

# Sort df by year and quarter
df=df.sort_values(['Year','Quarter'])

# Convert numeric cols to float
df = pd.concat([df[['Country','description_sector']], 
                df.drop(['Country','description_sector'],
                        axis=1).astype(float)], axis=1)

In [51]:
# Encode Quarter to preserve its cyclic nature (useful for ml tasks)

# Define the period for the trigonometric encoding (4 for quarters in a year)
period = 4
# Apply trigonometric coding
df['sin_quarter'] = np.sin(2 * np.pi * df['Quarter'] / period)
df['cos_quarter'] = np.cos(2 * np.pi * df['Quarter'] / period)

In [52]:
# Create missing indicator columns
for col in cols:
    missing_indicator_col_name = 'MI_'+col
    df[missing_indicator_col_name] = df[col].isnull().astype(int)

In [53]:
# To prevent leakage, we will output the data for task2 before any outlier and 
# missing data inputing technique
df.to_csv('output/0_clean_arff/df_task2.csv',index=False)

In [54]:
# Continuing for task1 data
# I dont know if this makes financial sense but ,I will remove massive outliers because they seem off
def remove_outliers_iqr(df,IQR_multiplier):
    """
    Remove outliers from each column of a DataFrame using the Interquartile Range (IQR) method.
    Turns the outliers to None.
    Smaller values of IQR_multiplier will remove outliers closer to the mean,
    larger values of IQR_multiplier will only remove outliers farther from the mean.
    A general rule of thumb is IQR_multiplier=1.5
    """
    df_no_outliers = df.copy()  # Create a copy of the DataFrame to avoid modifying the original
    
    # Select columns with float dtype
    float_cols = df.select_dtypes(include=['float']).columns

    # Iterate over each float column
    for column in df[float_cols].columns:
        # Calculate the first quartile (Q1) and third quartile (Q3) for the column
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        
        # Calculate the IQR for the column
        IQR = Q3 - Q1
        
        # Define the lower and upper bounds for outliers for the column
        lower_bound = Q1 - IQR_multiplier * IQR
        upper_bound = Q3 + IQR_multiplier * IQR
        
        # Replace outliers with None for the column
        df_no_outliers[column] = df[column].mask((df[column] < lower_bound) | (df[column] > upper_bound), other=None)
    
    return df_no_outliers

df = remove_outliers_iqr(df,4)

In [55]:
df = df.copy() # Defragment dataframe

In [56]:
df=df.sort_values(['Year','Quarter']).reset_index(drop=True)

In [57]:
# Inpute missing data with interpolation as it is time series data
df[cols] = df[cols].interpolate(method='linear')

In [58]:
# Inpute the remaining missing values (due to missing from the start or from the end of df)
imputer = KNNImputer(n_neighbors=5)
df[cols] = imputer.fit_transform(df[cols])

In [59]:
# As we are going to use linear regression, let s normalize the data
scaler = StandardScaler()
# Normalize
df[cols] = scaler.fit_transform(df[cols])

In [60]:
# Save the df_task1
df.to_csv('output/0_clean_arff/df_task1.csv',index=False)